# TER - Visualiser l'Alogorithme Ford Fulkson et Dijkstra avec Jupyter

### Tingting ZHU & Yu YANG

---

* Introduction
      1.   Contexte
      2.   Motivation et objectif
      3.   Introduction de Jupyter


* Etapes de réalisation
       1.  Algorithme de Ford-Fulkerson
       2.  Algorithme de Dijkstra
* Conclusion


---



# Contexte

Ce projet TER(Travaille d'Etude et de Recherche) est de faire algorithmique interactive avec Jupyter. Nous l'utilisons pour modéliser l'algorithme de Ford-Fulkerson et de Dijkstra.

# Motivation et objectif
Pendant le cours d'algorithme, on a pris beaucoup de algorithme pour trouver le plus court chemin, tel que Flot Max(Ford-Fulkerson) et Dijkstra. Afin de pouvoir visualiser le processus d'exécution de l'algorithme, nous programmons en Python avec Jupyter

# Introduction de Jupyter
Jupyter Notebook est une application Web qui facilite la création et le partage de documents de programmes. Il permet de exécuter les codes cellule par cellule et propose beaucoup de outils de visualisation.

# Réaliser en l'algorithme Flot Max(Ford-Fulkerson)

In [ ]:
import networkx as nx
from GraphAlgorithmPlayer import GraphAlgorithmPlayer
import copy

In [ ]:
variables=[
          {'name': 'G',          'type':'graph'                                     },
          {'name':'edge_liste_path',      'type':'edges', 'color': 'red', 'display':True   },
          {'name':'edge_labels',    'type':'edges', 'label': True,    'display': False },
          {'name':'node_labels',        'type':'nodes', 'label': True,    'display': False },
          {'name':'flow',        'type':'nodes',   'display': True },
          {'name':'flot_ajoute',        'type':'nodes',   'display': True }
          ]

In [ ]:
class Edge(object):
    def __init__(self, u, v, w):
        self.source = u
        self.sink = v  
        self.capacity = w
    def __repr__(self):
        return "%s->%s:%s" % (self.source, self.sink, self.capacity)

In [ ]:
class FlowNetwork(object):
    def __init__(self):
        self.adj = {}
        self.flow = {}
 
    def add_vertex(self, vertex):
        self.adj[vertex] = []
 
    def get_edges(self, v):
        return self.adj[v]
 
    def add_edge(self, u, v, w=0):
        if u == v:
            raise ValueError("u == v")
        edge = Edge(u,v,w)
        redge = Edge(v,u,0)
        edge.redge = redge
        redge.redge = edge
        self.adj[u].append(edge)
        self.adj[v].append(redge)
        self.flow[edge] = 0
        self.flow[redge] = 0
 
    def find_path(self, source, sink, path):
        if source == sink:
            return path
        for edge in self.get_edges(source):
            residual = edge.capacity - self.flow[edge]
            if residual > 0 and edge not in path:
                result = self.find_path( edge.sink, sink, path + [edge]) 
                if result != None:
                    return result
 
    def max_flow(self,G,source, sink): 
        pathG = {}
        path = self.find_path(source, sink, [])     
        while path != None:
            #print(path)
            residuals = [edge.capacity - self.flow[edge] for edge in path]
            flow = min(residuals)
            edge_liste_path = []
            edge_path=(0,0)
            for edge in path:
                color_path = []
                self.flow[edge] += flow
                self.flow[edge.redge] -= flow
                try:
                    G[edge.source][edge.sink]['flow']
                except KeyError:
                    dict_exists = False
                else:
                    dict_exists = True
                if(dict_exists):
                    G[edge.source][edge.sink]['flow']+=flow
                    edge_path = (edge.source,edge.sink)
                else:
                    G[edge.sink][edge.source]['flow']-=flow
                    edge_path = (edge.sink,edge.source)
                if(edge_path in edge_liste_path):
                    edge_liste_path.remove(edge_path)
                else : 
                    edge_liste_path.append(edge_path)

            path = self.find_path(source, sink, [])
            #npos=dict(zip(nodes,vnode))
            
            node_labels = dict()
            for node in G.nodes:
                node_labels[node] = str(node)
            edge_labels=dict()
            for edge in G.edges :
                edge_labels[edge]=str(G.edges[edge]['flow']) + ' / ' + str(G.edges[edge]['weight'])
            flot_ajoute = sum(self.flow[edge] for edge in self.get_edges(source))
            player.set_value(copy.deepcopy(locals()))
        return flot_ajoute

In [ ]:
g=FlowNetwork()

In [ ]:
file = open('dataFlot1.txt')
line = file.readline()
line = line.strip().split(' ')
print(line)
G = nx.DiGraph()
[G.add_node(v) for v in line]
[g.add_vertex(v) for v in line]
line = file.readline()
while line:
    line = line.strip().split(' ')
    G.add_edge(line[0],line[1],weight=int(line[2]),flow=0)
    g.add_edge(line[0],line[1],int(line[2]))
    line = file.readline()
file.close()

npos = nx.spring_layout(G)

node_labels = dict()
for node in G.nodes:
     node_labels[node] = str(node)
edge_labels= dict()
for edge in G.edges :
     edge_labels[edge]=str(G.edges[edge]['flow']) + ' / ' + str(G.edges[edge]['weight'])
     #edge_labels[edge]=str(edge)
#player.set_value(copy.deepcopy(locals()))

In [ ]:
player = GraphAlgorithmPlayer(G, variables=variables, view='networkx')
player

In [ ]:
g.max_flow(G, 's','t')

# Réaliser en l'algorithme Dijkstra

In [ ]:
from collections import defaultdict
from heapq import *
import networkx as nx
import matplotlib.pyplot as plt
from GraphAlgorithmPlayer import GraphAlgorithmPlayer
import copy


In [ ]:

variables=[
          {'name': 'G',          'type':'graph'                                     },
          {'name':'node_labels',        'type':'nodes', 'label': True,    'display': False },
          {'name':'non_developpe',        'type':'nodes', 'color': 'green',   'display': True },
          {'name':'developpe',        'type':'nodes', 'color': 'red',   'display': True },
          {'name':'v1',        'type':'nodes', 'color': 'yellow',   'display': True }
          ]

In [ ]:
def dijkstra_raw(G,edges, dep, dst):
    g = defaultdict(list)
    for u,v,p in edges:#u->v p:poids
        g[u].append((p,v))
    q, seen = [(0,dep,())], set()
    developpe = []

    while q:
        non_developpe = []
        (poid,v1,trajet) = heappop(q)
        if v1 not in seen:
            print('trajet',trajet)
            seen.add(v1)
            
            list_v1=[]  #list de trajet : red
            if len(trajet)>0:
                u = trajet[0]
                list_v1.append(u)    
                v = trajet[1]
                while len(v)>0:
                    u = v[0]
                    list_v1.append(u)    
                    v = v[1]
            
            trajet = (v1, trajet)
            developpe.append(v1)
            if v1 == dst:
                player.set_value(copy.deepcopy(locals()))
                return poid,trajet
            
            list_v2 = []
            for p, v2 in g.get(v1, ()):
                if v2 not in seen:
                    list_v2.append(v2)
                    heappush(q, (poid+p, v2, trajet))
            for i in q :
                non_developpe.append(i[1])
                    
            player.set_value(copy.deepcopy(locals()))
            
            
    return float("inf"),[]
 
def dijkstra(G,edges, dep, dst):
    pcc = -1    #pcc:plus court chemin
    ret_trajet=[]
    longueur,trajet_queue = dijkstra_raw(G,edges, dep, dst)
    if len(trajet_queue)>0:
        pcc = longueur    ## 1. Get the length firstly;
        ## 2. Decompose the path_queue, to get the passing nodes in the shortest path.
        u = trajet_queue[0]
        ret_trajet.append(u)    ## 2.1 Record the destination node firstly;
        v = trajet_queue[1]
        while len(v)>0:
            u = v[0]
            ret_trajet.append(u)    ## 2.2 Record other nodes, till the source-node.
            v = v[1]
        ret_trajet.reverse() ## 3. Reverse the list finally, to make it be normal sequence.
    return pcc,ret_trajet



In [ ]:

data =[('a','b',2),('a','c',3),('b','c',4),('b','d',3),('c','e',5),('d','f',2),('c','d',6),('f','g',4),('c','g',5)]

G = nx.DiGraph()#.random_regular_graph
G.add_weighted_edges_from(data)


In [ ]:
player = GraphAlgorithmPlayer(G, variables=variables, view='networkx')
player

In [ ]:
length,Shortest_path = dijkstra(G,data, 'a', 'g')
print ('length = ',length)
print ('The shortest path is ',Shortest_path)

# Conclusion
*  A traver la modélisations les deux algorithme avec Jupyter, nous avons famille avec des bibliothèque de Jupyter 
* Jupyter est plus simple pour tester les codes cellule par cellule
* Jupyter est bien adapté aux objectifs de ce TER(algorithmique interactive)